In [ ]:
library(Seurat)

# Generate new Seurat objects after running CellBender and Cell Ranger

In [ ]:
# # Run within scratch 
# cellbender.dir <- "/projects/mludwig/DVC/data/cellbender/mouse/Ludwig/"
# 
# cellranger.dir <- "/projects/mludwig/DVC/data/data-runs/mouse/Ludwig/"
# 
# prefix <- list.files(path=cellbender.dir, pattern=NULL, all.files=FALSE,
#                     full.names=FALSE)
# 
# suffix <- paste0(prefix, "_filtered.h5")
# 
# final.list <- vector(mode = "list", length = length(prefix))
# 
# for (i in 1:length(prefix)) {
#   
#   print(i)
# 
#   sample <- gsub(".*_", "", prefix[i])
# 
#   counts.path <- paste0(cellbender.dir, prefix[i], "/", suffix[i])
#   counts <- Read10X_h5(counts.path, use.names = T)
# 
#   counts.cellranger.path <- paste0(cellranger.dir, prefix[i], "/",
#                                    "outs", "/", "filtered_feature_bc_matrix")
#   counts.cellranger <- Read10X(counts.cellranger.path)
# 
# 
#   counts <- counts[, which(colnames(counts) %in% colnames(counts.cellranger))]
#   colnames(counts) <- paste0(sample, "_", colnames(counts))
#   colnames(counts) <- gsub("-[1-9]", "", colnames(counts)) # remove -1 after cell bar codes to match
# 
#   final.list[[i]] <- CreateSeuratObject(counts = counts)
#   final.list[[i]]$sample <- sample
#   final.list[[i]]$run <- sample
#   final.list[[i]]$hash.ID <- sample
# 
#   final.list[[i]]$treatment <- gsub("[0-9]+", "", sample)
#   final.list[[i]]$treatment <- gsub("W", "WM", final.list[[i]]$treatment)
#   final.list[[i]]$pool <- sample
# 
# 
#   idx <- which(apply(final.list[[i]]@assays$RNA@counts, 2, sum) == 0)
# 
# 
#   final.list[[i]] <- subset(final.list[[i]], cells = idx, invert = T)
# 
#   final.list[[i]][["percent.mt"]] <- PercentageFeatureSet(final.list[[i]],
#                                                           pattern = "^mt-")
# 
#   names(final.list)[i] <- sample
# 
# }
# 
# 
# saveRDS(final.list,
#          file = "/projects/mludwig/DVC/data/mouse_snRNA/Ludwig/seurat_obj_list.rds")

# Load new preprocessed Ludwig et al. data

In [ ]:
final.list <- readRDS("/projects/mludwig/DVC/data/mouse_snRNA/Ludwig/seurat_obj_list.rds")

# Filter data based on nUMIs and percent mitochondiral RNA
mouse.list <- vector(mode = "list", length = length(final.list))
names(mouse.list) <- names(final.list)

for (i in 1:length(mouse.list)) {
  mouse.list[[i]] <- subset(final.list[[i]], nCount_RNA > 1.5e+04 | nCount_RNA < 250 | percent.mt  > 2,
                            invert = T)
}

# Merge data

In [ ]:
mouse <- merge(x = mouse.list[[1]], y = mouse.list[-1])

# Normalize

In [ ]:
mouse <- SCTransform(mouse, verbose = F, method = "qpoisson")

# Run dimensionality reduction and clustering

In [ ]:
# PCA
VariableFeatures(mouse[["SCT"]]) <- rownames(mouse[["SCT"]]@scale.data)
mouse <- RunPCA(mouse, verbose = F, npcs = 100)

# Select number of PCs
ElbowPlot(mouse, ndims = 100)

# UMAP
mouse <- RunUMAP(mouse, dims = 1:30, n.neighbors = 50)

# Find clusters

In [ ]:
mouse <- FindNeighbors(mouse, dims = 1:30, k.param = 50, verbose = F)
mouse <- FindClusters(mouse, resolution = 0.1, verbose = F)
mouse <- FindClusters(mouse, resolution = 1, verbose = F)

# Save 

In [ ]:
mouse@meta.data <- mouse@meta.data[, c("nCount_RNA", "nFeature_RNA", 
                                       "nCount_SCT", "nFeature_SCT",
                                       "percent.mt", "pool", "hash.ID", 
                                       "treatment", "run", 
                                       "SCT_snn_res.0.1", "SCT_snn_res.1")]

saveRDS(mouse, file = "/projects/mludwig/DVC/output/Seurat_objs/mouse/mouse_Ludwig_Seurat_obj.rds")